### Importing required dependencies

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/debangshu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv(r'/home/debangshu/Downloads/Dataset/spam_ham_dataset.csv', engine= 'python')

In [5]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [6]:
def textClean(text):
    nopunc = ''.join([char for char in text if char not in string.punctuation])
    nopunc = set(nopunc.split())
    clean = ' '.join([word for word in nopunc if word.lower() not in stopwords.words('english')])
    return clean
#applied_df = df.copy()
df['text'] = df['text'].apply(lambda mail:textClean(mail))

In [7]:
# Train-Test split (0.7/0.3)
test_df = df.sample(frac = 0.3)
train_df = pd.concat([df, test_df]).drop_duplicates(keep=False)

In [ ]:
# Master list of all unique words in dataset
big_string = ''
for text in train_df['text']:
    big_string = ' '.join([big_string,text])

big_string = big_string.split()
big_string = [*set(big_string)]

In [ ]:
# Emails to vectors function
def word2vect(email, big_string=big_string):
    
    email = set(email.split())
    v = np.zeros(len(big_string))
    idx = [i for i,word in enumerate(big_string) if word in email]
    v[np.array(idx)] = 1
    return v

In [ ]:
# Matrix of all vectors
X_train = np.array([word2vect(text) for v2 in train_df['text']])
X_test = np.array([word2vect(text) for v2 in test_df['text']])

In [ ]:
# Output vector
y_train = np.array([n for n in train_df['v1']])
y_test = np.array([n for n in test_df['v1']])

In [ ]:
# spam = 1; ham = 0
# Finding probabilities (genarating model using X_test)
def prob_x_given_y(y_output,word):
    
    feature_given_output = X_train[:,word][np.where(y_train==y_output)] # Gives a particular word's column for a partucular y_output
    return sum(feature_given_output)/len(feature_given_output)

prob_x_given_spam = [prob_x_given_y(1, word) for word in range(len(big_string))]
prob_x_given_ham = [prob_x_given_y(0, word) for word in range(len(big_string))]
prob_x = [prob_x_given_y(y_train, word) for word in range(len(big_string))]
prob_spam = sum(y_train)/len(y_train)
prob_ham = 1-prob_spam

In [ ]:
# Training model and getting predictions for y_test
y_pred=[]
for text in X_test:
    
    class_prior_spam = prob_spam
    fl_spam = [prob_x_given_spam[i]+10**-18 for i,word in enumerate(text) if word==1]
    dec_spam = [Decimal(x) for x in fl_spam]
    likelihood_spam = np.prod(dec_spam)

    class_prior_ham = prob_ham
    l_ham = [prob_x_given_ham[i]+10**-18 for i,word in enumerate(text) if word==1]
    dec_ham = [Decimal(x) for x in fl_ham]
    likelihood_ham = np.prod(dec_ham)

    predictor_prior = likelihood_spam*Decimal(class_prior_spam)+likelihood_ham* Decimal(class_prior_ham)


    post_prob_spam = likelihood_spam*Decimal(class_prior_spam)/predictor_prior
    post_prob_ham = likelihood_ham*Decimal(class_prior_ham)/predictor_prior
    prediction = 1 if post_prob_spam>post_prob_ham else 0
    y_pred.append(prediction)

In [ ]:
# Confusion matrix function
def comp_confmat(y_train, y_pred):
    # extract the different classes
    classes = np.unique(y_train)
    # initialize the confusion matrix
    confmat = np.zeros((len(classes), len(classes)))
    # loop across the different combinations of actual / predicted classes
    for i in range(len(classes)):
        for j in range(len(classes)):
           # count the number of instances in each combination of actual / predicted classes
           confmat[i, j] = np.sum((y_train == classes[i]) & (y_pred== classes[j]))
    accuracy = (confmat[0,0]+confmat[1,1])/(confmat[0,0]+confmat[1,1]+confmat[0,1]+confmat[1,0])
    print(accuracy)
    return confmat

# confusion matrix
print(comp_confmat(y_test, y_pred))

In [ ]:
y_pred=[]
for text in X_train:
    class_prior_spam = prob_spam
    fl_spam = [prob_x_given_spam[i]+10**-18 for i,word in enumerate(text) if word==1]
    dec_spam = [Decimal(x) for x in fl_spam]
    likelihood_spam = np.prod(dec_spam)

    class_prior_ham = prob_ham
    fl_ham = [prob_x_given_ham[i]+10**-18 for i,word in enumerate(text) if word==1]
    dec_ham = [Decimal(x) for x in fl_ham]
    likelihood_ham = np.prod(dec_ham)

    predictor_prior = likelihood_spam*Decimal(class_prior_spam)+likelihood_ham* Decimal(class_prior_ham)


    post_prob_spam = likelihood_spam*Decimal(class_prior_spam)/predictor_prior
    post_prob_ham = likelihood_ham*Decimal(class_prior_ham)/predictor_prior
    prediction = 1 if post_prob_spam>post_prob_ham else 0
    y_pred.append(prediction)

# confusion matrix
print(comp_confmat(y_train, y_pred))